In [2]:
import hickle as hkl

In [3]:
import numpy as np
np.random.seed(1234)

In [4]:
PESeq = hkl.load('./Gen_data/11_Gen_Pro_Ehr.hkl')

## pre-process data set

In [5]:
X=[]
Y=[]
for index in range(len(PESeq['label'])):
    Y.append(PESeq['label'][index])
    X.append(np.append(PESeq['Pro-Kmer'][index],PESeq['Ehr-Kmer'][index],axis=1))
X = np.array(X)
Y  = np.array(Y)

In [10]:
print np.shape(X)
print np.shape(Y)
print  Y[:3]

(23752, 4096, 2)
(23752,)
['1' '1' '1']


In [6]:
VALIDATION_SPLIT =0.1

index = range(len(PESeq['label']))
np.random.shuffle(index)
nb_validation_samples = int(VALIDATION_SPLIT*len(Y))

In [7]:
x_train = X[index[: - nb_validation_samples]]
y_train = Y[index[:-nb_validation_samples]]
x_val = X[index[-nb_validation_samples:]]
y_val =Y[index[-nb_validation_samples:]]

In [8]:
from keras.utils.np_utils import to_categorical

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [9]:
y_train = to_categorical(y_train) #one-hot 编码labels
y_val = to_categorical(np.asarray(y_val))

# Make a Network

In [11]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout
from keras import layers

In [16]:
import os
os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])


In [17]:
kmer_input = Input(shape=(4096,2), dtype='float32')
l_lstm =Bidirectional(LSTM(50,return_sequences=True))(kmer_input)
l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
l_att_drop = Dropout(0.5)(l_att)
preds = Dense(len( y_train[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_att)
model  = Model (kmer_input,preds)

In [18]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()

model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 4096, 2)       0                                            
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 4096, 100)     21200       input_3[0][0]                    
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 4096, 100)     0           bidirectional_2[0][0]            
____________________________________________________________________________________________________
attlayer_1 (AttLayer)            (None, 100)           100         dropout_2[0][0]                  
____________________________________________________

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

Train on 21377 samples, validate on 2375 samples
Epoch 1/20
21377/21377 [==============================] - 2768s - loss: 0.4982 - acc: 0.5319 - val_loss: 0.4959 - val_acc: 0.5364
Epoch 2/20
21377/21377 [==============================] - 3158s - loss: 0.4975 - acc: 0.5387 - val_loss: 0.4963 - val_acc: 0.5103
Epoch 3/20
21377/21377 [==============================] - 3080s - loss: 0.4973 - acc: 0.5418 - val_loss: 0.4954 - val_acc: 0.5554
Epoch 4/20
21377/21377 [==============================] - 3162s - loss: 0.4972 - acc: 0.5453 - val_loss: 0.4954 - val_acc: 0.5482
Epoch 5/20
21377/21377 [==============================] - 3092s - loss: 0.4971 - acc: 0.5450 - val_loss: 0.4951 - val_acc: 0.5436
Epoch 6/20
21377/21377 [==============================] - 3917s - loss: 0.4969 - acc: 0.5458 - val_loss: 0.4953 - val_acc: 0.5263
Epoch 7/20
21377/21377 [==============================] - 4591s - loss: 0.4968 - acc: 0.5457 - val_loss: 0.4952 - val_acc: 0.5368
Epoch 8/20
21377/21377 [=================

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

Train on 21377 samples, validate on 2375 samples
Epoch 1/20
21377/21377 [==============================] - 4652s - loss: 0.4967 - acc: 0.5486 - val_loss: 0.4950 - val_acc: 0.5411
Epoch 2/20
21377/21377 [==============================] - 4579s - loss: 0.4966 - acc: 0.5460 - val_loss: 0.4951 - val_acc: 0.5528
Epoch 3/20
21377/21377 [==============================] - 4655s - loss: 0.4965 - acc: 0.5524 - val_loss: 0.4949 - val_acc: 0.5385
Epoch 4/20
21377/21377 [==============================] - 4581s - loss: 0.4965 - acc: 0.5485 - val_loss: 0.4964 - val_acc: 0.5133
Epoch 5/20
21377/21377 [==============================] - 4657s - loss: 0.4967 - acc: 0.5493 - val_loss: 0.4950 - val_acc: 0.5394
Epoch 6/20
21377/21377 [==============================] - 4580s - loss: 0.4970 - acc: 0.5447 - val_loss: 0.4956 - val_acc: 0.5512
Epoch 7/20
21377/21377 [==============================] - 4655s - loss: 0.4969 - acc: 0.5463 - val_loss: 0.4952 - val_acc: 0.5427
Epoch 8/20
21377/21377 [=================